---
# **ACTIVIDAD 1: Regresión logística**

---

Realizado por: Daniel Casañas

Fecha: 31 de marzo de 2023

---
# **1. Instalación**

---

### 1.1  Librerías

In [ ]:
!pip install mplcursors
!pip install pandas plotly
!pip install selenium
!pip install python-docx
!pip install python-Levenshtein
!pip install plotly
!pip install fuzzywuzzy

### 1.2  Módulos

In [18]:
# Importar bibliotecas

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
import seaborn as sns
import statsmodels.formula.api as smf
import math
import pandas as pd


# Importar Sub - Bibliotecas

from tabulate import tabulate
from scipy.stats import rankdata
from IPython.display import display,HTML
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from fuzzywuzzy import fuzz
from tabulate import tabulate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


---
# **2. Marco de datos**

---

In [14]:
# Importar el archivo CSV
df_V2 = pd.read_csv("/content/WC.csv")

# Imprime las primeras filas del DataFrame
df_V2.head()

,Class label,Alcohol,Malic Acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavonid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.60,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.20,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.60,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.80,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.00,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


### 2.1  Punto de Control

In [15]:
# Generar una copia del DataFrame con otro nombre
df_V3_copy = df_V2.copy()
# Guardar el DataFrame modificado en un archivo CSV con un nuevo nombre y extensión
df_V3_copy.to_csv("/content/WC_V3.csv", index=False)
# Importar el archivo CSV
df_V3 = pd.read_csv("/content/WC_V3.csv")

---
# **3. Regresión Logística**

---

In [16]:
# Agregar los nombres de las columnas
df_V3.columns = ['Class label', 'Alcohol', 'Malic Acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavonoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

# Convertir los valores de las columnas a numéricos
columnas_numericas = ['Alcohol', 'Malic Acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavonoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']
df_V3[columnas_numericas] = df_V3[columnas_numericas].apply(pd.to_numeric)

# Agregar dos decimales a los valores numéricos
pd.options.display.float_format = '{:.2f}'.format      

# Separar los datos en una matriz de características (X) y un vector objetivo (y)
X = df_V3.drop(['Class label'], axis=1)  # Se eliminan las columnas que no son el vector objetivo
y = df_V3['Class label']

# Imprimir la matriz de características (X) en formato de tabla
print("\n\033[43m\033[30m\033[1mMatriz de características (X):\033[0m")
print(X.head().to_string(index=False, header=True))

# Imprimir el vector objetivo (y) en formato de tabla
print("\n\033[43m\033[30m\033[1mVector objetivo (y):\033[0m")
print(y.head().to_string(index=False, header=True))


Matriz de características (X):
 Alcohol  Malic Acid  Ash  Alcalinity of ash  Magnesium  Total phenols  Flavanoids  Nonflavonoid phenols  Proanthocyanins  Color intensity  Hue  OD280/OD315 of diluted wines  Proline
   14.23        1.71 2.43              15.60        127           2.80        3.06                  0.28             2.29             5.64 1.04                          3.92     1065
   13.20        1.78 2.14              11.20        100           2.65        2.76                  0.26             1.28             4.38 1.05                          3.40     1050
   13.16        2.36 2.67              18.60        101           2.80        3.24                  0.30             2.81             5.68 1.03                          3.17     1185
   14.37        1.95 2.50              16.80        113           3.85        3.49                  0.24             2.18             7.80 0.86                          3.45     1480
   13.24        2.59 2.87              21.00        1

In [19]:
# Definir X_train y X_test
X = df_V3[['Alcohol', 'Malic Acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavonoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']]
y = df_V3['Class label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Definir las transformaciones
num_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# Definir las columnas que serán transformadas
num_features = ['Alcohol', 'Malic Acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavonoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

# Definir el preprocesamiento
preprocessor = ColumnTransformer(transformers=[('num', num_transformer, num_features)])

# Ajustar el preprocesamiento en el conjunto de entrenamiento
preprocessor.fit(X_train)

# Transformar los conjuntos de entrenamiento y prueba
X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Convertir a DataFrames de pandas
X_train_transformed = pd.DataFrame(X_train_transformed, columns=num_features)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=num_features)

# Mostrar los DataFrames resultantes
print("\033[43m\033[30m\033[1mRespuesta:\033[0m")
print("Conjunto de entrenamiento transformado (forma):", X_train_transformed.shape)
print(X_train_transformed.head())
print("Conjunto de prueba transformado (forma):", X_test_transformed.shape)
print(X_test_transformed.head())

Respuesta:
Conjunto de entrenamiento transformado (forma): (142, 13)
   Alcohol  Malic Acid   Ash  Alcalinity of ash  Magnesium  Total phenols  \
0     0.88        0.80  0.64               0.13       0.49          -0.70   
1    -0.37       -0.76 -0.40               0.33      -1.41          -1.44   
2    -1.70       -0.34 -0.32              -0.45      -0.15           1.25   
3     0.52        1.38  0.42               1.00       0.14          -0.75   
4     0.64       -0.51  0.90               0.13      -0.29           0.45   

   Flavanoids  Nonflavonoid phenols  Proanthocyanins  Color intensity   Hue  \
0       -1.43                  1.07            -1.37             0.35  0.03   
1       -0.50                  1.70             0.02            -0.84  0.03   
2        0.32                 -1.52            -0.43            -0.76  0.90   
3       -1.23                  0.52            -0.33             0.95 -1.11   
4        0.68                 -0.66             0.09            -0.64  0.

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear un modelo de regresión logística con regularización L1
model_L1 = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)

# Entrenar el modelo
model_L1.fit(X_train, y_train)

# Predecir las etiquetas de la clase en el conjunto de prueba
y_pred_L1 = model_L1.predict(X_test)

# Calcular la exactitud del modelo
accuracy_L1 = accuracy_score(y_test, y_pred_L1)

# Crear un modelo de regresión logística con regularización L2
model_L2 = LogisticRegression(penalty='l2', solver='liblinear', C=0.1)

# Entrenar el modelo
model_L2.fit(X_train, y_train)

# Predecir las etiquetas de la clase en el conjunto de prueba
y_pred_L2 = model_L2.predict(X_test)

# Calcular la exactitud del modelo
accuracy_L2 = accuracy_score(y_test, y_pred_L2)

print('Exactitud con regularización L1:', accuracy_L1)
print('Exactitud con regularización L2:', accuracy_L2)


Exactitud con regularización L1: 0.9629629629629629
Exactitud con regularización L2: 1.0


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear un modelo de regresión logística con regularización L1
model_L1 = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)

# Entrenar el modelo
model_L1.fit(X_train, y_train)

# Predecir las etiquetas de la clase en el conjunto de prueba
y_pred_L1 = model_L1.predict(X_test)

# Calcular la exactitud del modelo
accuracy_L1 = accuracy_score(y_test, y_pred_L1)

# Crear un modelo de regresión logística con regularización L2
model_L2 = LogisticRegression(penalty='l2', solver='liblinear', C=0.1)

# Entrenar el modelo
model_L2.fit(X_train, y_train)

# Predecir las etiquetas de la clase en el conjunto de prueba
y_pred_L2 = model_L2.predict(X_test)

# Calcular la exactitud del modelo
accuracy_L2 = accuracy_score(y_test, y_pred_L2)

print('Exactitud con regularización L1:', accuracy_L1)
print('Exactitud con regularización L2:', accuracy_L2)


Exactitud con regularización L1: 0.9629629629629629
Exactitud con regularización L2: 1.0
